In [3]:
import os
os.chdir('../streamlit')
import warnings
warnings.filterwarnings('ignore', category=Warning, module='streamlit')

In [4]:
# In your Jupyter notebook, load your TEG data however you normally do
# For example:
from utils import get_complete_teg_data
teg_data = get_complete_teg_data()

# Or if loading directly:
# teg_data = pd.read_parquet('path/to/your/data.parquet')
# # Then filter to completed TEGs only

In [10]:
import pandas as pd
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def create_teg_ranking_table(teg_data: pd.DataFrame, 
                           scoring_type: str, 
                           row_dimension: str = 'Player', 
                           col_dimension: str = 'TEG') -> pd.DataFrame:
    """
    Create ranking table for completed TEGs by scoring type.
    
    Parameters:
    - teg_data: DataFrame from get_complete_teg_data() or similar
    - scoring_type: 'Sc', 'GrossVP', 'NetVP', 'Stableford'  
    - row_dimension: 'Player', 'Pl' (for bonus feature)
    - col_dimension: 'TEG', 'TEGNum' (for bonus feature)
    
    Returns:
    - DataFrame with Players as rows, TEGs as columns, ranks as values
    """
    logger.info(f"Creating TEG ranking table for {scoring_type}")
    
    # Step 1: Create pivot table with totals by scoring type
    pivot_df = teg_data.pivot_table(
        index=row_dimension, 
        columns=col_dimension, 
        values=scoring_type, 
        aggfunc='sum',  # Sum scores across all rounds in each TEG
        fill_value=None  # Use None for TEGs where player didn't participate
    )
    
    # Step 1.5: Sort columns properly (TEG 1, TEG 2, ... not TEG 1, TEG 10, TEG 11, TEG 2...)
    if col_dimension == 'TEG':
        # Sort by extracting the numeric part from "TEG X" format
        def teg_sort_key(teg_name):
            try:
                return int(str(teg_name).replace('TEG ', ''))
            except (ValueError, AttributeError):
                return 999  # Put any non-standard names at the end
        
        sorted_columns = sorted(pivot_df.columns, key=teg_sort_key)
        pivot_df = pivot_df[sorted_columns]
        
    elif col_dimension == 'TEGNum':
        # Sort by TEGNum values directly (should be numeric)
        sorted_columns = sorted(pivot_df.columns)
        pivot_df = pivot_df[sorted_columns]
    
    # Step 2: Apply ranking logic (reuse existing ascending/descending logic)
    # Determine ranking order based on scoring type
    ascending = True  # Default for gross scores (lower is better)
    if scoring_type == 'Stableford':
        ascending = False  # Higher stableford is better
    
    # Step 3: Rank each TEG column independently
    ranked_df = pivot_df.copy()
    
    for teg_col in pivot_df.columns:
        # Get non-null values for ranking
        col_data = pivot_df[teg_col].dropna()
        
        if len(col_data) > 0:
            # Apply ranking (method='min' matches existing leaderboard logic)
            ranks = col_data.rank(method='min', ascending=ascending)
            
            # Handle ties with '=' suffix (matches existing logic)
            duplicated_scores = col_data.duplicated(keep=False)
            if duplicated_scores.any():
                tie_ranks = ranks[duplicated_scores].astype(str) + '='
                ranks[duplicated_scores] = tie_ranks
            else:
                ranks = ranks.astype(int)
            
            # Update the ranked dataframe
            ranked_df.loc[ranks.index, teg_col] = ranks
    
    # Step 4: Clean up the output
    ranked_df = ranked_df.reset_index()
    
    # Sort by player name for consistent display
    ranked_df = ranked_df.sort_values(by=row_dimension)
    
    logger.info(f"TEG ranking table created with {len(ranked_df)} players and {len(ranked_df.columns)-1} TEGs")
    
    return ranked_df


In [13]:
# Test gross vs par rankings
grossvp_rankings = create_teg_ranking_table(teg_data, 'GrossVP')
print("Gross vs Par Rankings:")
print(grossvp_rankings)

# Test stableford rankings
stableford_rankings = create_teg_ranking_table(teg_data, 'Stableford') 
print("\nStableford Rankings:")
print(stableford_rankings)

# Test with different dimensions
pl_rankings = create_teg_ranking_table(teg_data, 'NetVP', 
                                     row_dimension='Pl', 
                                     col_dimension='TEGNum')
print("\nPlayer Code + TEG Number:")
print(pl_rankings)

Gross vs Par Rankings:
TEG          Player  TEG 2  TEG 3  TEG 4  TEG 5  TEG 6  TEG 7  TEG 8  TEG 9  \
0        Alex BAKER    NaN    NaN    NaN    NaN    NaN    6.0    6.0    6.0   
1      David MULLIN    1.0    1.0    1.0    1.0    1.0    1.0    1.0    1.0   
2    Gregg WILLIAMS    5.0    5.0    5.0    5.0    5.0    5.0    3.0    4.0   
3      Henry MELLER    3.0    4.0    3.0    4.0    4.0    3.0    4.0    NaN   
4    John PATTERSON    NaN    NaN    NaN    NaN    NaN    NaN    NaN    3.0   
5         Jon BAKER    4.0    3.0    4.0    2.0    2.0    4.0    2.0    2.0   
6    Stuart NEUMANN    2.0    2.0    2.0    3.0    3.0    2.0    5.0    5.0   

TEG  TEG 10  TEG 11  TEG 12  TEG 13  TEG 14  TEG 15  TEG 16  TEG 17  
0       4.0     4.0     5.0     4.0     4.0     6.0     5.0     5.0  
1       1.0     2.0     2.0     3.0     1.0     3.0     2.0     2.0  
2       3.0     3.0     3.0     2.0     3.0     1.0     1.0     3.0  
3       NaN     NaN     NaN     NaN     NaN     NaN     NaN     

C:\Users\JBA33\AppData\Local\Temp\ipykernel_29796\3428616999.py:73: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['3.0=' '3.0=']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  ranks[duplicated_scores] = tie_ranks
C:\Users\JBA33\AppData\Local\Temp\ipykernel_29796\3428616999.py:78: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[5.0 2.0 '3.0=' 1.0 '3.0=']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  ranked_df.loc[ranks.index, teg_col] = ranks
C:\Users\JBA33\AppData\Local\Temp\ipykernel_29796\3428616999.py:73: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['2.0=' '2.0=']' has dtype incompatible with float64, please explicitly cast to a compatible dtype f

In [9]:
print(stableford_rankings)

TEG          Player  TEG 10  TEG 11  TEG 12 TEG 13  TEG 14  TEG 15  TEG 16  \
0        Alex BAKER     1.0     2.0     5.0   3.0=     2.0     6.0     5.0   
1      David MULLIN     3.0     5.0     6.0    5.0     1.0     5.0     3.0   
2    Gregg WILLIAMS     2.0     3.0     2.0    2.0     3.0     1.0     2.0   
3      Henry MELLER     NaN     NaN     NaN    NaN     NaN     NaN     NaN   
4    John PATTERSON     6.0     4.0     1.0   3.0=     NaN     3.0     NaN   
5         Jon BAKER     4.0     1.0     4.0    1.0     4.0     4.0     4.0   
6    Stuart NEUMANN     5.0     NaN     3.0    NaN     NaN     2.0     1.0   

TEG TEG 17  TEG 2  TEG 3  TEG 4 TEG 5  TEG 6 TEG 7  TEG 8  TEG 9  
0     3.0=    NaN    NaN    NaN   NaN    NaN   6.0    2.0    2.0  
1      2.0    1.0    2.0    1.0   5.0    4.0  2.0=    4.0    3.0  
2      5.0    4.0    3.0    5.0   2.0    1.0  2.0=    1.0    5.0  
3      NaN    5.0    4.0    3.0  3.0=    5.0   1.0    5.0    NaN  
4     3.0=    NaN    NaN    NaN   NaN   